<a href="https://colab.research.google.com/github/hajjihi/IndabaxMorocco/blob/master/Convnet_Interpretability_IndabaXMorocco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Overview of Interpretability Approaches in Deep Learning** ![Texte alternatif…](https://raw.githubusercontent.com/hajjihi/IndabaxMorocco/master/logoIndabaX.png)


Author:** Pr Hajji Hicham (PhD in CS)**, School of Geomatic Sciences and Surveying Engineering, IAV H2

First presented at: IndabaxMorocco 2019, Deep Learning Conference

April 30, 2019

Contact: hajjihi@gmail.com, linkedin: https://www.linkedin.com/in/dr-hajji-hicham-6601606/ Twitter: @hajjihi



Ce code se veut comme un récap de plusieurs approches d'interprétations des ConvNets inplémentées dans des librairies comme Keras-Vis, LIME, Google Lucid, DeepExplain. 



Il s'inspire des colab et notebooks suivants:
- https://github.com/raghakot/keras-vis
- https://github.com/marcotcr/lime/
- https://github.com/tensorflow/lucid
- https://github.com/marcoancona/DeepExplain
- https://colab.research.google.com/github/idealo/cnn-exposed


Colab Execution Config: **Python3 & GPU** 

#Clone du Github Directory

In [0]:
!rm -rf IndabaxMorocco

!git clone https://github.com/hajjihi/IndabaxMorocco.git

# **Exploration de Keras-vis**

## ** Installation & parametrage**

> Installation des packages nécessaires









In [0]:
#installation de Keras-vis pour la visualisation des attributions Saliency, GradCAM, ...
# site https://raghakot.github.io/keras-vis/
!pip install git+https://github.com/raghakot/keras-vis.git -U


In [0]:
#!pip uninstall matplotlib

In [0]:
#!pip install matplotlib===3.0.0

### **Utility functions**

### **Import des packages**

In [0]:
# import packages
import os
import pathlib
import requests

import numpy as np
import matplotlib.image as mpimg

from PIL import Image
from io import BytesIO
from skimage import feature, transform
from matplotlib.pyplot import figure
from matplotlib import pyplot as plt
%matplotlib inline

# import keras dependencies
from keras.models import Model
from keras.applications import MobileNet as CNN
from keras.applications.mobilenet import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.utils import to_categorical
import keras.backend as K

# import specific functions from keras-vis package
from vis.utils import utils
from vis.visualization import visualize_cam, overlay

### **Import des poids du modèle VGG16**

![Texte alternatif…](https://image.slidesharecdn.com/fttl-keras-nov2016-161120190859/95/transfer-learning-and-fine-tuning-for-cross-domain-image-classification-with-keras-8-638.jpg?cb=1479669021)

In [0]:
from keras.applications import VGG16
from vis.utils import utils
from keras import activations

# Construire le modèle VGG16 avec les poids Imagenet
model = VGG16(weights='imagenet', include_top=True)

##Préparation du Modèle et des images

### **Préparation du modèle**

In [0]:
# remplacer la fonction softmax de la dernière couche avec une fonction linéaire (softmax pose problème dans le gradient de classe en question)

layer_idx = utils.find_layer_idx(model, 'predictions')

model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)


### **Préparation des images**#

In [0]:
from vis.utils import utils
from matplotlib import pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (18, 6)

img1 = utils.load_img('IndabaxMorocco/ouzel1.jpg', target_size=(224, 224))
#img2 = utils.load_img('IndabaxMorocco/ouzel2.jpg', target_size=(224, 224))

f, ax = plt.subplots(1, 1)
#f, ax = plt.subplots(1, 2)

ax.imshow(img1)

#ax[1].imshow(img2)

In [0]:
loaded_image = np.array(image.load_img('IndabaxMorocco/ouzel1.jpg', target_size=(224, 224)))
processed_image = preprocess_input(loaded_image)
preds = model.predict(processed_image[np.newaxis, :])
preds_name = decode_predictions(preds, top=5)
preds_name

## Affichage des attributions avec Keras-vis

![Texte alternatif…](http://i.imgur.com/bHLF8it.jpg)

### **Saliency Maps**# 


In [0]:
#voir ici les codes des classes dans imagenet: https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a
from vis.visualization import visualize_saliency, overlay
from vis.utils import utils
from keras import activations

layer_idx = utils.find_layer_idx(model, 'predictions')
f, ax = plt.subplots(1, 1)
ax.imshow(img1)
#ax[0].imshow(img1)
#ax[1].imshow(img2)
f, ax = plt.subplots(1, 1)
for i, img in enumerate([img1]):    
    # 20 is the imagenet index corresponding to `ouzel`
    grads = visualize_saliency(model, layer_idx, filter_indices=20, seed_input=img)
    
    # visualiser grads comme une heatmap (carte de chaleur)
    ax.imshow(grads, cmap='jet')

### ** Guided Backpropagation**# 


In [0]:
#Selon le site de Kears.vis:
#le paramètre "guided": Modifies backprop to only propagate positive gradients for positive activations.
#References: Details on guided back propagation can be found in paper: [String For Simplicity: The All Convolutional Net] (https://arxiv.org/pdf/1412.6806.pdf)

#le paramètre "relu": Modifies backprop to only propagate positive gradients.
#References: Details can be found in the paper: [Visualizing and Understanding Convolutional Networks] (https://arxiv.org/pdf/1311.2901.pdf)
plt.figure()
f, ax = plt.subplots(1, 1)
ax.imshow(img1)
#ax[1].imshow(img2)
for modifier in ['guided']:
    f, ax = plt.subplots(1, 1)
    plt.suptitle(modifier)
    for i, img in enumerate([img1]):    
        # 20 is the imagenet index corresponding to `ouzel`
        # 669: 'mosquito net',
        # 141: 'redshank
        grads = visualize_saliency(model, layer_idx, filter_indices=141, 
                                   seed_input=img, backprop_modifier=modifier)
        # Lets overlay the heatmap onto original image.    

        ax.imshow(grads, cmap='jet')

### ** Grad-CAM** 



![Texte alternatif…](https://camo.githubusercontent.com/450498bd998fd99d51b647d2b6c8631e94585522/687474703a2f2f692e696d6775722e636f6d2f4a614762645a352e706e67)

*** Affichage du Modèle***

In [0]:
model.summary()

In [0]:
import numpy as np
import matplotlib.cm as cm
from vis.visualization import visualize_cam
plt.figure()
f, ax = plt.subplots(1, 1)
ax.imshow(img1)
#ax[1].imshow(img2)
for modifier in ['guided']:
    f, ax = plt.subplots(1, 1)
    plt.suptitle("vanilla" if modifier is None else modifier)
    for i, img in enumerate([img1]):    
        # 20 is the imagenet index corresponding to `ouzel`
        grads = visualize_cam(model, layer_idx, filter_indices=20, 
                              seed_input=img, backprop_modifier=modifier)        
        # Lets overlay the heatmap onto original image.    
        jet_heatmap = np.uint8(cm.jet(grads)[..., :3] * 255)
                
        #print(grads.shape) 
        
        ax.imshow(overlay(jet_heatmap, img))
        #ax[i].imshow(overlay(grads, img))

## Activation Maximization avec Keras Vis

![Texte alternatif…](https://image.slidesharecdn.com/dlcv2018d3l4interpretability-180702081944/95/interpretability-of-convolutional-neural-networks-eva-mohedano-upc-barcelona-2018-29-638.jpg?cb=1531320625)

###Préparation du Modèle

In [0]:
from keras.applications import VGG16
from vis.utils import utils
from keras import activations

model = VGG16(weights='imagenet', include_top=True)


layer_idx = utils.find_layer_idx(model, 'predictions')

model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)

###Affichage de l'activation

In [0]:
from vis.visualization import visualize_activation

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (18, 6)
from vis.input_modifiers import Jitter

#

# 20 est le code de la catégorie 'ouzel' dans imagenet
#pour plus d'infos sur les paramétres, merci de voir le site : https://raghakot.github.io/keras-vis/
#liste des codes de classe dans imagenet: https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a
img = visualize_activation(model, layer_idx, filter_indices=96, max_iter=500, input_modifiers=[Jitter(16)])
plt.imshow(img)

#tester d'autres classes depuis https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a
#96: 'toucan'
#120: 'fiddler crab'



# Exploration de LIME

 Local Interpretable Model-Agnostic Explanations 
![Texte alternatif…](https://blogs.infosupport.com/wp-content/uploads/2018/07/lime.png)

##Préparation

In [0]:
#les sources codes proviennent du notebook https://github.com/marcotcr/lime/blob/master/doc/notebooks/Tutorial%20-%20Image%20Classification%20Keras.ipynb
#https://github.com/marcotcr/lime/tree/master/doc/notebooks
!pip install lime --quiet


In [0]:
import os
import keras
from keras.applications import inception_v3 as inc_net
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
print('Notebook run using keras:', keras.__version__)

In [0]:
inet_model = inc_net.InceptionV3()


In [0]:
#Une fonction utilitaire pour transformer l'image utilisée
def transform_img_fn(path_list):
    out = []
    for img_path in path_list:
        img = image.load_img(img_path, target_size=(299, 299))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = inc_net.preprocess_input(x)
        out.append(x)
    return np.vstack(out)

##Prédiction 

In [0]:
images = transform_img_fn([os.path.join('IndabaxMorocco/','scorpion.jpg')])
#images = transform_img_fn([os.path.join('IndabaxMorocco/','cat_mouse.jpg')])

plt.imshow(images[0] / 2 + 0.5)
preds = inet_model.predict(images)
for x in decode_predictions(preds)[0]:
    print(x)

##Explication

![Texte alternatif…](https://cdn-images-1.medium.com/max/1600/1*mdnqGE3TWHbg266B9vzgOA.jpeg)

In [0]:
%load_ext autoreload
%autoreload 2
import os,sys
try:
    import lime
except:
    sys.path.append(os.path.join('..', '..')) # add the current directory
    import lime
from lime import lime_image

explainer = lime_image.LimeImageExplainer()
#%%time
# Création des instances perturbées : nombre 1000 et création de la courbe de régression pondérée pour trouver la perturbation qui a la plus grande pondération
explanation = explainer.explain_instance(images[0], inet_model.predict, top_labels=5, hide_color=0, num_samples=1000)

Regardons l'explication de la classe Top : **scorpion**

In [0]:
#('n01770393', 'scorpion', 0.82200426)
#('n02105251', 'briard', 0.0011861428)
#('n02112706', 'Brabancon_griffon', 0.0010143266)
#('n03290653', 'entertainment_center', 0.0009968795)
#('n01692333', 'Gila_monster', 0.00084664806)

from skimage.segmentation import mark_boundaries

temp, mask = explanation.get_image_and_mask(71, positive_only=True, num_features=15, hide_rest=True, min_weight=0.01)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

Ou avec le reste de l'image

In [0]:
temp, mask = explanation.get_image_and_mask(71, positive_only=True, num_features=15, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

Nous pouvons rajouter les pros et les cons de la classification scorpion (pros en vert, contres en rouge)

In [0]:
temp, mask = explanation.get_image_and_mask(71, positive_only=False, num_features=15, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))


#Exploration de Google Lucid 

depuis: https://github.com/tensorflow/lucid

##Feature Invertion

Reference:  [Understanding Deep Image Representations by Inverting Them](https://arxiv.org/pdf/1412.0035.pdf), Mahendran & Vedaldi, 2015

![Texte alternatif…](https://camo.githubusercontent.com/d131a299baffb375706cf20c17530feed307ca08/68747470733a2f2f73746f726167652e676f6f676c65617069732e636f6d2f6c756369642d7374617469632f6d6973632f737469636b6572732f636f6c61622d666561747572652d696e76657273696f6e2e6970796e622e706e67)

In [0]:
# Install Lucid
!pip install --quiet lucid==0.0.5

In [0]:
# Import libraries
import numpy as np
import tensorflow as tf
import scipy.ndimage as nd

from google.colab import files

import lucid.modelzoo.vision_models as models
import lucid.optvis.objectives as objectives
import lucid.optvis.param as param
import lucid.optvis.render as render
import lucid.optvis.transform as transform
from lucid.misc.io import show, load
from lucid.misc.io.reading import read

# Import the InceptionV1 (GoogLeNet) model from the Lucid modelzoo

model = models.InceptionV1()
model.load_graphdef()

def imgToModelSize(arr):
  W = model.image_shape[0]
  w, h, _ = arr.shape
  s = float(W) / min(w,h)
  arr = nd.zoom(arr, [s, s, 1], mode="nearest")
  w, h, _ = arr.shape
  dw, dh = (w-W)//2, (h-W)//3
  return arr[dw:dw+W, dh:dh+W]

@objectives.wrap_objective
def dot_compare(layer, batch=1, cossim_pow=0):
  def inner(T):
    dot = tf.reduce_sum(T(layer)[batch] * T(layer)[0])
    mag = tf.sqrt(tf.reduce_sum(T(layer)[0]**2))
    cossim = dot/(1e-6 + mag)
    return dot * cossim ** cossim_pow
  return inner

@objectives.wrap_objective
def dot_compare(layer, batch=1, cossim_pow=0):
  def inner(T):
    dot = tf.reduce_sum(T(layer)[batch] * T(layer)[0])
    mag = tf.sqrt(tf.reduce_sum(T(layer)[0]**2))
    cossim = dot/(1e-6 + mag)
    return dot * cossim ** cossim_pow
  return inner


In [0]:
def feature_inversion(img=None, layer=None, n_steps=512, cossim_pow=0.0):
  with tf.Graph().as_default(), tf.Session() as sess:
    img = imgToModelSize(img)
    
    objective = objectives.Objective.sum([
        1.0 * dot_compare(layer, cossim_pow=cossim_pow),
        objectives.blur_input_each_step(),
    ])

    t_input = tf.placeholder(tf.float32, img.shape)
    param_f = param.image(img.shape[0], decorrelate=True, fft=True, alpha=False)
    param_f = tf.stack([param_f[0], t_input])

    transforms = [
      transform.pad(8, mode='constant', constant_value=.5),
      transform.jitter(8),
      transform.random_scale([0.9, 0.95, 1.05, 1.1] + [1]*4),
      transform.random_rotate(list(range(-5, 5)) + [0]*5),
      transform.jitter(2),
    ]

    T = render.make_vis_T(model, objective, param_f, transforms=transforms)
    loss, vis_op, t_image = T("loss"), T("vis_op"), T("input")

    tf.global_variables_initializer().run()
    for i in range(n_steps): _ = sess.run([vis_op], {t_input: img})

    result = t_image.eval(feed_dict={t_input: img})
    show(result[0])

##Execution sur toutes les couches

In [0]:
img = load("https://storage.googleapis.com/lucid-static/building-blocks/examples/dog_cat.png")


figure(figsize=(10, 10), dpi=100)
imgplot = mpimg.imread('https://storage.googleapis.com/lucid-static/building-blocks/examples/dog_cat.png')
plt.imshow(imgplot)
plt.grid(False)
plt.axis('off')
plt.show()
    


#layers = ['conv2d%d' % i for i in range(0, 3)] + ['mixed3a', 'mixed3b', 
#                                                  'mixed4a', 'mixed4b', 'mixed4c', 'mixed4d', 'mixed4e',
#                                                  'mixed5a', 'mixed5b']
layers = ['conv2d%d' % i for i in range(0, 3)] + ['mixed3a', 'mixed3b']

In [0]:
for layer in layers:
  print(layer)
  feature_inversion(img, layer=layer)
  print ()

#Exploration de Deep Explain


##Installation et prédiction

In [0]:
#Installation de DeepExplain  pour utiliser LRP 
# A unified framework of perturbation and gradient-based attribution methods for Deep Neural Networks interpretability. 
!pip install git+https://github.com/marcoancona/DeepExplain.git#egg=deepexplain

###Fonctions utiles

In [0]:
#Quelques fonctions proviennent de https://colab.research.google.com/github/idealo/cnn-exposed/blob/master/notebooks/Attribution.ipynb#scrollTo=U0rHlH4AnQPV
from deepexplain.tensorflow import DeepExplain
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tempfile, sys, os
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
from scipy.misc import imread
import tensorflow as tf
from tensorflow.contrib.slim.nets import inception

slim = tf.contrib.slim
    
from deepexplain.tensorflow import DeepExplain

from skimage import feature, transform
import numpy as np
import matplotlib.pyplot as plt

# import keras dependencies
from keras.models import Model
from keras.applications import MobileNet as CNN
from keras.applications.mobilenet import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.utils import to_categorical
import keras.backend as K


# import specific functions from keras-vis package
from vis.utils import utils
from vis.visualization import visualize_cam, overlay

    
def plot_single_image(image_path, fig_size=(10, 10), dpi=100):
    figure(figsize=fig_size, dpi=dpi)
    img = mpimg.imread(image_path)
    plt.imshow(img)
    plt.grid(False)
    plt.axis('off')
    plt.show()
    

# Plotting function for saliency maps
def plot_custom(data, xi=None, cmap='RdBu_r', axis=plt, percentile=100, dilation=3.0, alpha=0.8):
    dx, dy = 0.05, 0.05
    xx = np.arange(0.0, data.shape[1], dx)
    yy = np.arange(0.0, data.shape[0], dy)
    xmin, xmax, ymin, ymax = np.amin(xx), np.amax(xx), np.amin(yy), np.amax(yy)
    extent = xmin, xmax, ymin, ymax
    cmap_xi = plt.get_cmap('Greys_r')
    cmap_xi.set_bad(alpha=0)
    overlay = None
    if xi is not None:
        # Compute edges (to overlay to heatmaps later)
        xi_greyscale = xi if len(xi.shape) == 2 else np.mean(xi, axis=-1)
        in_image_upscaled = transform.rescale(xi_greyscale, dilation, mode='constant')
        edges = feature.canny(in_image_upscaled).astype(float)
        edges[edges < 0.5] = np.nan
        edges[:5, :] = np.nan
        edges[-5:, :] = np.nan
        edges[:, :5] = np.nan
        edges[:, -5:] = np.nan
        overlay = edges

    abs_max = np.percentile(np.abs(data), percentile)
    abs_min = -abs_max

    if len(data.shape) == 3:
        data = np.mean(data, 2)
    axis.imshow(data, extent=extent, interpolation='bicubic', cmap=cmap, vmin=abs_min, vmax=abs_max)
    if overlay is not None:
        axis.imshow(overlay, extent=extent, interpolation='bicubic', cmap=cmap_xi, alpha=alpha)
    axis.axis('off')
    return axis


def plot_comparison(target_image_path, map_array, title=''):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(26, 20))

    img_orig = Image.open(target_image_path).resize((224, 224))
    xi = (map_array[0,:] - np.min(map_array[0,:]))
    xi /= np.max(xi)

    ax = axes.flatten()[0]
    ax.imshow(img_orig)
    ax.set_title('Original', fontdict={'fontsize': 20})
    ax.axis('off')

    plot_custom(attributions[0], xi = xi, axis=axes[1], dilation=.5, percentile=99, alpha=.2).set_title(title, fontdict={'fontsize': 20})
    plt.show()

    #from https://raw.githubusercontent.com/marcoancona/DeepExplain/master/examples/utils.py


def plot(data, xi=None, cmap='RdBu_r', axis=plt, percentile=100, dilation=3.0, alpha=0.8):
    dx, dy = 0.05, 0.05
    xx = np.arange(0.0, data.shape[1], dx)
    yy = np.arange(0.0, data.shape[0], dy)
    xmin, xmax, ymin, ymax = np.amin(xx), np.amax(xx), np.amin(yy), np.amax(yy)
    extent = xmin, xmax, ymin, ymax
    cmap_xi = plt.get_cmap('Greys_r')
    cmap_xi.set_bad(alpha=0)
    overlay = None
    if xi is not None:
        # Compute edges (to overlay to heatmaps later)
        xi_greyscale = xi if len(xi.shape) == 2 else np.mean(xi, axis=-1)
        in_image_upscaled = transform.rescale(xi_greyscale, dilation, mode='constant')
        edges = feature.canny(in_image_upscaled).astype(float)
        edges[edges < 0.5] = np.nan
        edges[:5, :] = np.nan
        edges[-5:, :] = np.nan
        edges[:, :5] = np.nan
        edges[:, -5:] = np.nan
        overlay = edges

    abs_max = np.percentile(np.abs(data), percentile)
    abs_min = abs_max

    if len(data.shape) == 3:
        data = np.mean(data, 2)
    axis.imshow(data, extent=extent, interpolation='none', cmap=cmap, vmin=-abs_min, vmax=abs_max)
    if overlay is not None:
        axis.imshow(overlay, extent=extent, interpolation='none', cmap=cmap_xi, alpha=alpha)
    axis.axis('off')
    return axis

###Prédiction

In [0]:

path_resources = pathlib.Path('IndabaxMorocco/')
target_image = os.path.join(path_resources, 'snake.png')
plot_single_image(target_image)

model = CNN(include_top=True)

loaded_image = np.array(image.load_img(target_image, target_size=(224, 224)))
processed_image = preprocess_input(loaded_image)
preds = model.predict(processed_image[np.newaxis, :])
preds_name = decode_predictions(preds, top=3)
preds_name


##Création des cartes d'attributions

In [0]:
# import deepexplain to draw saliency maps
from deepexplain.tensorflow import DeepExplain
top_idx = preds.argsort()[::-1],


In [0]:
# import deepexplain to draw saliency maps
from deepexplain.tensorflow import DeepExplain
# Get saliency map
# Refer the API documentation for using deepexplain as below
with DeepExplain(session=K.get_session()) as de:
    model = CNN(include_top=True)
    input_tensor = model.layers[0].input
    fModel = Model(inputs=input_tensor, outputs = model.layers[-1].output)
    target_tensor = fModel(input_tensor)
    top_idx = preds.argsort()[::-1]
    ys = to_categorical(top_idx, num_classes=1000)
    xs = np.tile(processed_image, (1, 1, 1, 1))
    attributions = {
            'Saliency'         :de.explain('saliency', fModel.outputs[0] * ys, fModel.inputs[0], xs),
            'Occlusion [15x15]':    de.explain('occlusion', fModel.outputs[0] * ys, fModel.inputs[0], xs, window_shape=(15,15,3), step=4),
            'Gradient * Input':     de.explain('grad*input', fModel.outputs[0] * ys, fModel.inputs[0], xs),
            'Integrated Gradients': de.explain('intgrad', fModel.outputs[0] * ys, fModel.inputs[0], xs),
            'Epsilon-LRP':          de.explain('elrp', fModel.outputs[0] * ys, fModel.inputs[0], xs),
   }  

##Affichage des résultats

In [0]:
# Plot attributions
#from utils import plot, plt
#import matplotlib as mpl
#import matplotlib.pyplot as plt
%matplotlib inline

n_cols = int(len(attributions)) + 1
n_rows = len(xs) 
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(3*n_cols, 3*n_rows))

for i, xi in enumerate(xs):
    xi = (xi - np.min(xi))
    xi /= np.max(xi)
    ax = axes.flatten()[i*n_cols]
    ax.imshow(xi)
    ax.set_title('Original')
    ax.axis('off')
    for j, a in enumerate(attributions):
        axj = axes.flatten()[i*n_cols + j + 1]
        plot(attributions[a][i], xi = xi, axis=axj, dilation=.5, percentile=99, alpha=.2).set_title(a)

A partir du notebook de site officiel Lucid https://colab.research.google.com/github/tensorflow/lucid/blob/master/notebooks/tutorial.ipynb

# Miscellaneous

In [0]:
model.summary()

In [0]:
from vis.visualization import get_num_filters

# The name of the layer we want to visualize
# You can see this in the model definition.
layer_name = 'block1_conv1'
layer_idx = utils.find_layer_idx(model, layer_name)

# Visualize all filters in this layer.
filters = np.arange(get_num_filters(model.layers[layer_idx]))

# Generate input image for each filter.
vis_images = []
for idx in filters:
    img = visualize_activation(model, layer_idx, filter_indices=idx)
    
    # Utility to overlay text on image.
    img = utils.draw_text(img, 'Filter {}'.format(idx))    
    vis_images.append(img)

# Generate stitched image palette with 8 cols.
stitched = utils.stitch_images(vis_images, cols=8)    
plt.axis('off')
plt.imshow(stitched)
plt.title(layer_name)
plt.show()